<a href="https://colab.research.google.com/github/yoonandmoon/GS_team/blob/cleansing-jw/news06(gm_ver)01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import warnings
import pandas as pd
warnings.filterwarnings(action = 'ignore')

In [33]:
# df2에 필요정보만 저장
# 콘텐트 \n, \t,\r 제거
# 데이트 날짜만 분리
df = pd.read_csv("news_data06.csv")

df2 = df[['title','content','date']]
df2['content'] = df['content'].str.replace('\n',' ').replace('\t',' ').replace('\r', ' ')
df2['date'] = df['date'].str.split(' ').str[0]

In [41]:
# 토큰화
from konlpy.tag import Komoran
import re
tokenizer = Komoran()

df2['morphsC'] = None
df2['morphsT'] = None

for i, row in df.iterrows():
    df2['morphsC'][i] = ' '.join(tokenizer.morphs(row['content']))
    df2['morphsT'][i] = ' '.join(tokenizer.morphs(row['title']))

# 한글 문자만 남기고 나머지 문자 제거
df2['morphsC'] = df2['morphsC'].apply(lambda x: re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', '', x))
df2['morphsT'] = df2['morphsT'].apply(lambda x: re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', '', x))

In [ ]:
dfTEst = df2[['title', 'morphsT']]
dfTEst

In [37]:
import spacy

# spaCy의 NLP 모델을 로드합니다.
nlp = spacy.load('en_core_web_sm')

In [31]:
def extract_words(text):
    # 텍스트를 spaCy의 NLP 모델을 사용하여 토큰화합니다.
    doc = nlp(text)

    # 길이가 2글자 이상이고, 명사(Noun), 동사(Verb), 형용사(Adjective)인 단어를 선택합니다.
    selected_words = [token.text for token in doc if len(token.text) >= 2 and token.pos_ in ['NOUN', 'VERB', 'ADJ']]

    # 선택된 단어를 공백으로 구분된 문자열로 결합합니다.
    return ' '.join(selected_words)

In [38]:
df2['morphsC'] = df2['morphsC'].apply(extract_words)
df2['morphsT'] = df2['morphsT'].apply(extract_words)

df3 = df2[['date','morphsT', 'morphsC']]

In [ ]:
df3